In [8]:
# set dataset name and run
# The output accuracy files for both majority voting and average probabilities
# are exported to the folder DSF Random Forest/dataset/
# the last line in both files contains the best variant of RF_d_tT where d is in {5,10,15,20}, and T in [2,25]
# The outputs of training process are exported to the folder DSF Random Forest/dataset/DetailsFiles/
dataset='spambase'

import csv,operator,sys,os
import numpy as np
import sklearn
import json
import FeatureGenerators.ReadData as ReadData
import time
from functools import reduce
import numpy as np


sys.path.append('arch-forest/data/adult/')
sys.path.append('arch-forest/data/bank/')
sys.path.append('arch-forest/data/wine-quality/')
sys.path.append('arch-forest/data/')
sys.path.append('arch-forest/code/')
import trainForest
import Tree


filesPath = "../frequentTreesInRandomForests/arch-forest/data/"
frequentTreesPath = "../frequentTreesInRandomForests/forests/rootedFrequentTrees/"
resultsPath = "../frequentTreesInRandomForests/DSF Random Forest/"


#import train data
from FeatureGenerators.ReadData import readDataAdult,readWine, readWineTest

if (dataset == 'adult'):
    X_train,Y_train = ReadData.readDataAdultNormal('train')
    X_test,Y_test = ReadData.readDataAdultNormal('test')
if (dataset == 'spambase'):
    X_train,Y_train = ReadData.readDataSpambase('train')
    X_test,Y_test = ReadData.readDataSpambase('test')
if (dataset == 'letter'):
    X_train,Y_train = ReadData.readDataLetter('train')
    X_test,Y_test = ReadData.readDataLetter('test')    
if (dataset == 'wine-quality'):
        X_train,Y_train = ReadData.readWine()
        #X_test,Y_test = ReadData.readWineTest()



# Classification
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.estimator_checks import check_estimator
from sklearn.metrics import accuracy_score
    
import FeatureGenerators.DecisionSnippetFeatures

results_list = []
time_list = []
scoring_function = 'accuracy'
pattern_max_size=6
variant = 'NoLeafEdges'
countsList =[]

predictions_results_majority_vote = []
predictions_results_average_probability = []
best_prediction_majority_vote = 0
best_prediction_majority_vote_dsf = ''
best_prediction_average_probability = 0
best_prediction_average_probability_dsf = ''

for rf_depth in (5,10,15,20):
    for frequency in range(2,26):

    
    
        rootedFrequentTrees = "RF_"+str(rf_depth)+"_t"+str(frequency)
        f = open(frequentTreesPath+dataset+'/'+variant+'/leq'+str(pattern_max_size)+'/'+rootedFrequentTrees+'.json')
        frequentpatterns = json.load(f)
        f.close()

        dsf = FeatureGenerators.DecisionSnippetFeatures.FrequentSubtreeFeatures(map(lambda x: x['pattern'], frequentpatterns[-200:])) 

        fts = dsf.fit_transform(X_train,0)



        class_labels_list = ''
        class_labels_list_P = ''

        one_counter=0
        zero_counter=0
        max_id = 0
        max_class_id = 0
        trees_count = len(fts[0])

        for i in range(0,len(fts)):
            if (Y_train[i] > max_class_id):
                max_class_id +=1


            for j in range(0,len(fts[i])):
                if (fts[i][j] > max_id):
                    max_id = fts[i][j]
                  
        id_count= max_id + 1            
        class_count = max_class_id + 1  

        for i in range(0,class_count):
            class_labels_list += 'Y = '+str(i)+','
            class_labels_list_P += 'P(Y) = '+str(i)+','
    
            
          
        

        Matrix = np.zeros((trees_count,id_count,class_count))


        for i in range(0,len(fts)):
            for j in range(0,trees_count):

                     Matrix[j][fts[i][j]][Y_train[i]] +=1

                            
 


        
        Prefered_Class_Matrix = np.zeros((trees_count,id_count))
        Prefered_Class_Matrix_P = np.zeros((trees_count,id_count,class_count))
        f= open(resultsPath+'/'+dataset+'/Details_Files/'+'DSF_as_RF_'+str(rf_depth)+'_t'+str(frequency)+'_'+dataset+'_n.csv',"w")
        f.write('Feature ,Leaf id,'+class_labels_list+' Preferred Class, '+class_labels_list_P+'\n')
        for j in range(0,trees_count):
            for k in range(0,id_count):
                f.write(str(j)+','+str(k)+',')
                for c in range(0,class_count):
                        f.write(str(Matrix[j][k][c])+',')
                Prefered_Class_Matrix[j][k] = np.argmax(Matrix[j][k])       
                f.write(str(np.argmax(Matrix[j][k]))+',')  
                for c in range(0,class_count):
                    if (np.sum(Matrix[j][k]) != 0):
                        f.write(str(round(Matrix[j][k][c]/np.sum(Matrix[j][k]),3))+',')
                        Prefered_Class_Matrix_P[j][k][c] = round(Matrix[j][k][c]/np.sum(Matrix[j][k]),3)
                    else:
                        f.write('0,')
                f.write('\n')    

        f.close()            
        
        
        fts_test = dsf.fit_transform(X_test,0)



        labels_array = [0] * class_count
        labels_array_p = [0] * class_count
        predictedClass = 0
        correctPredictions = 0
        predictedClass_p = 0
        correctPredictions_p = 0

        for i in range(0,len(fts_test)):
            labels_array = [0] * class_count
            for j in range(0,trees_count):
                for c in range(0,class_count):

                    if (Prefered_Class_Matrix[j][fts_test[i][j]] == c):
                        labels_array[c] += 1

                        labels_array_p[c] += Prefered_Class_Matrix_P[j][fts_test[i][j]][c] 

            predictedClass =np.argmax(labels_array)
            predictedClass_p =np.argmax(labels_array_p)

            if (predictedClass == Y_test[i]):
                correctPredictions += 1
            labels_array = [0] * class_count
            if (predictedClass_p == Y_test[i]):
                correctPredictions_p += 1
            labels_array_p = [0] * class_count

        print(correctPredictions)
        
        print(correctPredictions/len(fts_test))
        predictions_results_majority_vote.append(correctPredictions/len(fts_test))
        predictions_results_average_probability.append(correctPredictions_p/len(fts_test))
        
        if (correctPredictions/len(fts_test) > best_prediction_majority_vote):
            best_prediction_majority_vote = correctPredictions/len(fts_test)
            best_prediction_majority_vote_dsf = 'RF_'+str(rf_depth)+'_t'+str(frequency)
            
        if (correctPredictions_p/len(fts_test) > best_prediction_average_probability):
            best_prediction_average_probability = correctPredictions_p/len(fts_test)
            best_prediction_average_probability_dsf = 'RF_'+str(rf_depth)+'_t'+str(frequency)    
        
        
f= open(resultsPath+'/'+dataset+'/'+'DSF_as_RF_accuracy_'+dataset+'_majority_vote'+'.csv',"w")
f.write('DSF,accuracy,\n')
counter = 0
for rf_depth in (5,10,15,20):
    for frequency in range(2,26):
        f.write('RF_'+str(rf_depth)+'_t'+str(frequency)+','+str(predictions_results_majority_vote[counter])+','+'\n')
        counter += 1
f.write('Best ,'+best_prediction_majority_vote_dsf+','+str(best_prediction_majority_vote)+',\n')
f.close()


f= open(resultsPath+'/'+dataset+'/'+'DSF_as_RF_accuracy_'+dataset+'_average_probability'+'.csv',"w")
f.write('DSF,accuracy,\n')
counter = 0
for rf_depth in (5,10,15,20):
    for frequency in range(2,26):
        f.write('RF_'+str(rf_depth)+'_t'+str(frequency)+','+str(predictions_results_average_probability[counter])+','+'\n')
        counter += 1
f.write('Best ,'+best_prediction_average_probability_dsf+','+str(best_prediction_average_probability)+',\n')
f.close()

710
0.6168549087749783
735
0.6385751520417029
708
0.6151172893136403
838
0.7280625543006082
850
0.738488271068636
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
707
0.6142484795829714
705
0.6125108601216334
720
0.6255430060816681
719
0.6246741963509991
691
0.6003475238922676
691
0.6003475238922676
894
0.7767158992180713
894
0.7767158992180713
894
0.7767158992180713
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.6099044309296264
702
0.609904

In [7]:
import numpy as np
class_labels_list = ''
class_labels_list_P = ''

one_counter=0
zero_counter=0
max_id = 0
max_class_id = 0
trees_count = len(fts[0])

for i in range(0,len(fts)):
    if (Y_train[i] > max_class_id):
        max_class_id +=1

        
    for j in range(0,len(fts[i])):
        if (fts[i][j] > max_id):
            max_id = fts[i][j]
                  
id_count= max_id + 1            
class_count = max_class_id + 1  

for i in range(0,class_count):
    class_labels_list += 'Y = '+str(i)+','
    class_labels_list_P += 'P(Y) = '+str(i)+','
    
            
          
print(class_count)  
print(id_count)

Matrix = np.zeros((trees_count,id_count,class_count))


for i in range(0,len(fts)):
    for j in range(0,trees_count):
       
             Matrix[j][fts[i][j]][Y_train[i]] +=1

                            

2
3


In [8]:
print(fts[0]) 
Prefered_Class_Matrix = np.zeros((trees_count,id_count))
Prefered_Class_Matrix_P = np.zeros((trees_count,id_count,class_count))
f= open(resultsPath+'/'+dataset+'/'+'DSF_as_RF_'+str(rf_depth)+'_t'+str(frequency)+'_'+dataset+'_n.csv',"w")
f.write('Feature ,Leaf id,'+class_labels_list+' Preferred Class, '+class_labels_list_P+'\n')
for j in range(0,trees_count):
    for k in range(0,id_count):
        f.write(str(j)+','+str(k)+',')
        for c in range(0,class_count):
                f.write(str(Matrix[j][k][c])+',')
        Prefered_Class_Matrix[j][k] = np.argmax(Matrix[j][k])       
        f.write(str(np.argmax(Matrix[j][k]))+',')  
        for c in range(0,class_count):
            if (np.sum(Matrix[j][k]) != 0):
                f.write(str(round(Matrix[j][k][c]/np.sum(Matrix[j][k]),3))+',')
                Prefered_Class_Matrix_P[j][k][c] = round(Matrix[j][k][c]/np.sum(Matrix[j][k]),3)
            else:
                f.write('0,')
        f.write('\n')    
                
f.close()            
        
        

[1 1 2 1 1 2 1 1 2 2 1 1 2 2 1 1 1 1 0 1 1]


In [11]:
j = 69
print(Matrix[j])

zero_counter = 0
one_counter = 0
two_counter = 0
three_counter = 0
four_counter = 0
for i in range(0,len(fts)):
    if (fts[i][j] == 0):
        zero_counter += 1
    if (fts[i][j] == 1):
        one_counter += 1
    if (fts[i][j] == 2):
        two_counter += 1
    if (fts[i][j] == 3):
        three_counter += 1
    if (fts[i][j] == 4):
        four_counter += 1    
print(zero_counter)
print(one_counter)
print(two_counter)
print(three_counter)
print(four_counter)
print(zero_counter + one_counter + two_counter + three_counter + four_counter)

[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [789. 766. 658. 805. 754. 581. 773. 722. 755. 747. 673. 761. 749. 745.
  753. 802. 771. 758. 744. 410. 427. 117. 635. 731. 191. 733.]
 [  0.   0.  78.   0.  14. 194.   0.  12.   0.   0.  66.   0.  43.  38.
    0.   1.  12.   0.   4. 386. 386. 647. 117.  56. 595.   1.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
0
17350
2650
0
0
20000


In [9]:
fts_test = dsf.fit_transform(X_test,0)

In [7]:

correct_predictions = 0
for i in range(0,len(Y_test)): 
    for j in range(0,trees_count):
        for k in range(0,id_count):
            if (Prefered_Class_Matrix[j][k] == Y_test[i]):
                correct_predictions += 1

print(correct_predictions)
                

3160431


In [11]:

labels_array = [0] * class_count
predictedClass = 0
correctPredictions = 0

for i in range(0,len(fts_test)):
    labels_array = [0] * class_count
    for j in range(0,trees_count):
        for c in range(0,class_count):
        
            if (Prefered_Class_Matrix[j][fts_test[i][j]] == c):
                #labels_array[c] += 1
                
                labels_array[c] += Prefered_Class_Matrix_P[j][fts_test[i][j]][c] 
    
    predictedClass =np.argmax(labels_array)
    
    if (predictedClass == Y_test[i]):
        correctPredictions += 1
    labels_array = [0] * class_count

print(correctPredictions)
print(correctPredictions/len(fts_test))


782
0.6794092093831451


In [18]:
counter = 0
for i in range(0,len(Y_train)):
    if (Y_train[i] == 1):
        counter += 1
   
print(counter)
print(len(Y_train))


1813
4601


In [193]:
for j in range(0,trees_count):
    for k in range(0,id_count):
        print(Matrix[j][k])
        print(Prefered_Class_Matrix[j][k])
       
        

[0. 0.]
0.0
[24550.  6315.]
0.0
[ 170. 1526.]
1.0
[0. 0.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[24440.  7128.]
0.0
[280. 713.]
1.0
[0. 0.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[8521.  377.]
0.0
[16199.  7464.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[6987. 2878.]
0.0
[17733.  4963.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[24707.  7831.]
0.0
[13. 10.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[23436.  7755.]
0.0
[1284.   86.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[5196.  387.]
0.0
[19524.  7454.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[23776.  7804.]
0.0
[944.  37.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[24613.  7535.]
0.0
[107. 306.]
1.0
[0. 0.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[11918.  1245.]
0.0
[12802.  6596.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[20740.  7378.]
0.0
[3980.  463.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[23068.  7650.]
0.0
[1652.  191.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[23443.  7521.]
0.0
[1277.  320.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[0. 0.]
0.0
[23761.  7775.]
0.0
[959.  66.]
0.

In [30]:
def getFeatureVectorAdult(entries):
	x = []
	x.append(float(entries[0])) # age = continous

	workclass = [0 for i in range(8)]
	if entries[1] == "Private":
		workclass[0] = 1
	elif entries[1] == "Self-emp-not-inc":
		workclass[1] = 1
	elif entries[1] == "Self-emp-inc":
		workclass[2] = 1
	elif entries[1] == "Federal-gov":
		workclass[3] = 1
	elif entries[1] == "Local-gov":
		workclass[4] = 1
	elif entries[1] == "State-gov":
		workclass[5] = 1
	elif entries[1] == "Without-pay":
		workclass[6] = 1
	else: #Never-worked
		workclass[7] = 1
	x.extend(workclass)
	x.append(float(entries[2]))

	education = [0 for i in range(16)]
	if entries[3] == "Bachelors":
		education[0] = 1
	elif entries[3] == "Some-college":
		education[1] = 1
	elif entries[3] == "11th":
		education[2] = 1
	elif entries[3] == "HS-grad":
		education[3] = 1
	elif entries[3] == "Prof-school":
		education[4] = 1
	elif entries[3] == "Assoc-acdm":
		education[5] = 1
	elif entries[3] == "Assoc-voc":
		education[6] = 1
	elif entries[3] == "9th":
		education[7] = 1
	elif entries[3] == "7th-8th":
		education[8] = 1
	elif entries[3] == "12th":
		education[9] = 1
	elif entries[3] == "Masters":
		education[10] = 1
	elif entries[3] == "1st-4th":
		education[11] = 1
	elif entries[3] == "10th":
		education[12] = 1
	elif entries[3] == "Doctorate":
		education[13] = 1
	elif entries[3] == "5th-6th":
		education[14] = 1
	if entries[3] == "Preschool":
		education[15] = 1

	x.extend(education)
	x.append(float(entries[4]))

	marital = [0 for i in range(7)]
	if entries[5] == "Married-civ-spouse":
		marital[0] = 1
	elif entries[5] == "Divorced":
		marital[1] = 1
	elif entries[5] == "Never-married":
		marital[2] = 1
	elif entries[5] == "Separated":
		marital[3] = 1
	elif entries[5] == "Widowed":
		marital[4] = 1
	elif entries[5] == "Married-spouse-absent":
		marital[5] = 1
	else:
		marital[6] = 1
	x.extend(marital)

	occupation = [0 for i in range(14)]
	if entries[6] == "Tech-support":
		occupation[0] = 1
	elif entries[6] == "Craft-repair":
		occupation[1] = 1
	elif entries[6] == "Other-service":
		occupation[2] = 1
	elif entries[6] == "Sales":
		occupation[3] = 1
	elif entries[6] == "Exec-managerial":
		occupation[4] = 1
	elif entries[6] == "Prof-specialty":
		occupation[5] = 1
	elif entries[6] == "Handlers-cleaners":
		occupation[6] = 1
	elif entries[6] == "Machine-op-inspct":
		occupation[7] = 1
	elif entries[6] == "Adm-clerical":
		occupation[8] = 1
	elif entries[6] == "Farming-fishing":
		occupation[9] = 1
	elif entries[6] == "Transport-moving":
		occupation[10] = 1
	elif entries[6] == "Priv-house-serv":
		occupation[11] = 1
	elif entries[6] == "Protective-serv":
		occupation[12] = 1
	else:
		occupation[13] = 1
	x.extend(occupation)

	relationship = [0 for i in range(6)]
	if entries[7] == "Wife":
		relationship[0] = 1
	elif entries[7] == "Own-child":
		relationship[1] = 1
	elif entries[7] == "Husband":
		relationship[2] = 1
	elif entries[7] == "Not-in-family":
		relationship[3] = 1
	elif entries[7] == "Other-relative":
		relationship[4] = 1
	else:
		relationship[5] = 1
	x.extend(relationship)

	race = [0 for i in range(5)]
	if entries[8] == "White":
		race[0] = 1
	elif entries[8] == "Asian-Pac-Islander":
		race[1] = 1
	elif entries[8] == "Amer-Indian-Eskimo":
		race[2] = 1
	elif entries[8] == "Other":
		race[3] = 1
	else:
		race[4] = 1
	x.extend(race)

	if (entries[9] == "Male"):
		x.extend([1,0])
	else:
		x.extend([0,1])

	x.append(float(entries[10]))
	x.append(float(entries[11]))
	x.append(float(entries[12]))

	native = [0 for i in range(42)]
	if entries[14] == "United-States":
		native[1] = 1
	elif entries[14] == "Cambodia":
		native[2] = 1
	elif entries[14] == "England":
		native[3] = 1
	elif entries[14] == "Puerto-Rico":
		native[4] = 1
	elif entries[14] == "Canada":
		native[5] = 1
	elif entries[14] == "Germany":
		native[6] = 1
	elif entries[14] == "Outlying-US(Guam-USVI-etc)":
		native[7] = 1
	elif entries[14] == "India":
		native[8] = 1
	elif entries[14] == "Japan":
		native[9] = 1
	elif entries[14] == "Greece":
		native[10] = 1
	elif entries[14] == "South":
		native[11] = 1
	elif entries[14] == "China":
		native[12] = 1
	elif entries[14] == "Cuba":
		native[13] = 1
	elif entries[14] == "Iran":
		native[14] = 1
	elif entries[14] == "Honduras":
		native[15] = 1
	elif entries[14] == "Philippines":
		native[16] = 1
	elif entries[14] == "Italy":
		native[17] = 1
	elif entries[14] == "Poland":
		native[18] = 1
	elif entries[14] == "Jamaica":
		native[19] = 1
	elif entries[14] == "Vietnam":
		native[20] = 1
	elif entries[14] == "Mexico":
		native[21] = 1
	elif entries[14] == "Portugal":
		native[22] = 1
	elif entries[14] == "Ireland":
		native[23] = 1
	elif entries[14] == "France":
		native[24] = 1
	elif entries[14] == "Dominican-Republic":
		native[25] = 1
	elif entries[14] == "Laos":
		native[26] = 1
	elif entries[14] == "Ecuador":
		native[27] = 1
	elif entries[14] == "Taiwan":
		native[28] = 1
	elif entries[14] == "Haiti":
		native[29] = 1
	elif entries[14] == "Columbia":
		native[30] = 1
	elif entries[14] == "Hungary":
		native[31] = 1
	elif entries[14] == "Guatemala":
		native[32] = 1
	elif entries[14] == "Nicaragua":
		native[33] = 1
	elif entries[14] == "Scotland":
		native[34] = 1
	elif entries[14] == "Thailand":
		native[35] = 1
	elif entries[14] == "Yugoslavia":
		native[36] = 1
	elif entries[14] == "El-Salvador":
		native[37] = 1
	elif entries[14] == "Trinadad&Tobago":
		native[38] = 1
	elif entries[14] == "Peru":
		native[39] = 1
	elif entries[14] == "Hong":
		native[40] = 1
	else:
		native[41] = 1

	return x

In [35]:

path = "/home/falkhoury/Study/Lab/Project/frequentTreesInRandomForests/arch-forest/data/"
X = []
Y = []       

f = open(path + "adult/adult.train")
counter = 0
for row in f:
    
    if (len(row) > 1):
        entries = row.replace("\n", "").replace(" ", "").split(",")
        x = getFeatureVectorAdult(entries)

        if (entries[-1] == "<=50K." or entries[-1] == "<=50K"):
            counter += 1
            if (counter %3 == 0):
                y = 0
                Y.append(y)
                X.append(x) 
                
                
        else:
                y = 1
                Y.append(y)
                X.append(x) 

        
X = np.array(X).astype(dtype=np.int32)
Y = np.array(Y)
f.close()
counter_zero = 0
counter_one = 0
for i in range(0,len(X)):
    if (Y[i] == 0):
        counter_zero += 1
    if (Y[i] == 1):
        counter_one += 1
print(counter_zero)
print(counter_one)    

5598
5374


449
1151
